In [1]:
!pip install datasets

In [2]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset, Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, Trainer, default_data_collator, RobertaTokenizer
from torch.utils.data import DataLoader
import numpy as np
import nltk

# Ensure necessary downloads for BLEU metric
nltk.download('punkt')

2024-08-15 10:03:52.486417: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 10:03:52.486475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 10:03:52.487844: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Load the CodeSearchNet dataset for each language
dataset_java = load_dataset('code_search_net', 'java')['train']
dataset_python = load_dataset('code_search_net', 'python')['train']
dataset_go = load_dataset('code_search_net', 'go')['train']

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/454451 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/26909 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15328 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/317832 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14291 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/14242 [00:00<?, ? examples/s]

In [11]:
# Function to sample 100 examples
def sample_data(dataset, num_samples=10000):
    sampled_indices = random.sample(range(len(dataset)), num_samples)
    return dataset.select(sampled_indices)

# Sample 100 pairs from each language
sampled_java = sample_data(dataset_java)
sampled_python = sample_data(dataset_python)
sampled_go = sample_data(dataset_go)

# Function to split data into train, validation, and test sets
def train_val_test_split_data(dataset, val_size=0.1, test_size=0.1):
    train_size = 1 - (val_size + test_size)
    train_dataset = dataset.shuffle(seed=42).select(range(int(train_size * len(dataset))))
    temp_dataset = dataset.shuffle(seed=42).select(range(int(train_size * len(dataset)), len(dataset)))
    split = int(len(temp_dataset) * (test_size / (test_size + val_size)))
    val_dataset = temp_dataset.select(range(len(temp_dataset) - split))
    test_dataset = temp_dataset.select(range(len(temp_dataset) - split, len(temp_dataset)))
    return train_dataset, val_dataset, test_dataset

# Split the data into train, validation, and test sets for each language
train_java, val_java, test_java = train_val_test_split_data(sampled_java)
train_python, val_python, test_python = train_val_test_split_data(sampled_python)
train_go, val_go, test_go = train_val_test_split_data(sampled_go)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')

# Preprocess function
def preprocess_function(examples):
    inputs = examples['func_code_string']
    targets = examples['func_documentation_string']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

# Preprocess the datasets
train_java = train_java.map(preprocess_function, batched=True)
val_java = val_java.map(preprocess_function, batched=True)
test_java = test_java.map(preprocess_function, batched=True)

train_python = train_python.map(preprocess_function, batched=True)
val_python = val_python.map(preprocess_function, batched=True)
test_python = test_python.map(preprocess_function, batched=True)

train_go = train_go.map(preprocess_function, batched=True)
val_go = val_go.map(preprocess_function, batched=True)
test_go = test_go.map(preprocess_function, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, TrainingArguments, default_data_collator
import nltk

# Ensure necessary downloads for BLEU metric
nltk.download('punkt')

# Custom collate function to handle different data types
def custom_collate_fn(features):
    batch = {}
    for k in features[0].keys():
        if isinstance(features[0][k], list) and isinstance(features[0][k][0], int):  # Tokenized data (assumed to be lists of integers)
            batch[k] = torch.tensor([f[k] for f in features])
        elif isinstance(features[0][k], torch.Tensor):  # Already tensors
            batch[k] = torch.stack([f[k] for f in features])
        else:
            batch[k] = [f[k] for f in features]  # Keep as a list (likely for non-tensor data like strings)
    
    return batch

# EWC Loss function
class EWCLoss(nn.Module):
    def __init__(self, model, old_params, fisher_matrix, lambda_=0.4):
        super(EWCLoss, self).__init__()
        self.model = model
        self.old_params = old_params
        self.fisher_matrix = fisher_matrix
        self.lambda_ = lambda_
    
    def forward(self, current_loss):
        ewc_loss = 0
        for name, param in self.model.named_parameters():
            if name in self.fisher_matrix:
                ewc_loss += torch.sum(self.fisher_matrix[name] * (param - self.old_params[name]).pow(2))
        total_loss = current_loss + self.lambda_ * ewc_loss
        return total_loss

def compute_fisher_information(model, dataloader, device):
    fisher_matrix = {}
    for name, param in model.named_parameters():
        fisher_matrix[name] = torch.zeros_like(param)

    model.eval()
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        model.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        for name, param in model.named_parameters():
            fisher_matrix[name] += param.grad.pow(2)

    for name in fisher_matrix:
        fisher_matrix[name] = fisher_matrix[name] / len(dataloader)

    return fisher_matrix

def save_old_params(model):
    old_params = {}
    for name, param in model.named_parameters():
        old_params[name] = param.clone().detach()
    return old_params

# Function to fine-tune the model with EWC
def fine_tune_with_ewc(model, train_dataset, val_dataset, output_dir, old_params=None, fisher_matrix=None):
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,  # Reduced batch size to fit in memory
        per_device_eval_batch_size=4,   # Reduced batch size to fit in memory
        evaluation_strategy="epoch",
        save_steps=1000,
        save_total_limit=2,
        logging_dir='./logs',
        gradient_accumulation_steps=2,  # Accumulate gradients to simulate larger batch size
        fp16=True,  # Enable mixed precision training to save memory
    )
    
    optimizer = optim.AdamW(model.parameters(), lr=5e-5)

    if old_params and fisher_matrix:
        ewc_loss = EWCLoss(model, old_params, fisher_matrix, lambda_=0.4)

    # Use the custom collate function
    train_dataloader = DataLoader(train_dataset, batch_size=training_args.per_device_train_batch_size, collate_fn=custom_collate_fn)

    for epoch in range(training_args.num_train_epochs):
        model.train()
        model.to(device)  # Ensure model is on the correct device
        torch.cuda.empty_cache()  # Clear cache to free up memory
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}  # Ensure batch is on the correct device
            outputs = model(**batch)
            loss = outputs.loss

            if old_params and fisher_matrix:
                loss = ewc_loss(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return model

# Function to evaluate the model using BLEU score
def evaluate_model(model, test_dataset, device):
    model.to(device)  # Ensure model is on the correct device
    torch.cuda.empty_cache()  # Clear cache to free up memory
    test_results = []
    
    for example in test_dataset:
        inputs = tokenizer(example['func_code_string'], return_tensors="pt", max_length=512, truncation=True).to(device)
        outputs = model.generate(**inputs)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        reference = example['func_documentation_string']
        test_results.append({"prediction": prediction, "reference": reference})
    
    predictions = [result["prediction"] for result in test_results]
    references = [nltk.word_tokenize(result["reference"]) for result in test_results]
    tokenized_predictions = [nltk.word_tokenize(pred) for pred in predictions]
    
    bleu = nltk.translate.bleu_score.corpus_bleu([[ref] for ref in references], tokenized_predictions)
    
    return {"bleu": bleu}

# Define the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained CodeT5 model
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')

# Step 1: Fine-tune on Java
model = fine_tune_with_ewc(model, train_java, val_java, './results_java')
fisher_matrix_java = compute_fisher_information(model, DataLoader(val_java, batch_size=4, collate_fn=custom_collate_fn), device)
old_params_java = save_old_params(model)

# Step 2: Fine-tune on Python with EWC
model = fine_tune_with_ewc(model, train_python, val_python, './results_python', old_params_java, fisher_matrix_java)
fisher_matrix_python = compute_fisher_information(model, DataLoader(val_python, batch_size=4, collate_fn=custom_collate_fn), device)
old_params_python = save_old_params(model)

# Step 3: Fine-tune on Go with EWC
model = fine_tune_with_ewc(model, train_go, val_go, './results_go', old_params_python, fisher_matrix_python)

# Evaluate the model
java_results_after_go = evaluate_model(model, test_java, device)
python_results_after_go = evaluate_model(model, test_python, device)
go_results_after_go = evaluate_model(model, test_go, device)

# Display results
print("Results after training on Java, Python, and Go with EWC:")
print("Java Test Results:", java_results_after_go)
print("Python Test Results:", python_results_after_go)
print("Go Test Results:", go_results_after_go)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Results after training on Java, Python, and Go with EWC:
Java Test Results: {'bleu': 0.0032440345624518857}
Python Test Results: {'bleu': 0.0111070966258134}
Go Test Results: {'bleu': 0.09627148154406495}


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, TrainingArguments, default_data_collator
import nltk

# Ensure necessary downloads for BLEU metric
nltk.download('punkt')

# Custom collate function to handle different data types
def custom_collate_fn(features):
    batch = {}
    for k in features[0].keys():
        if isinstance(features[0][k], list) and isinstance(features[0][k][0], int):  # Tokenized data (assumed to be lists of integers)
            batch[k] = torch.tensor([f[k] for f in features])
        elif isinstance(features[0][k], torch.Tensor):  # Already tensors
            batch[k] = torch.stack([f[k] for f in features])
        else:
            batch[k] = [f[k] for f in features]  # Keep as a list (likely for non-tensor data like strings)
    
    return batch

# EWC Loss function
class EWCLoss(nn.Module):
    def __init__(self, model, old_params, fisher_matrix, lambda_=0.4):
        super(EWCLoss, self).__init__()
        self.model = model
        self.old_params = old_params
        self.fisher_matrix = fisher_matrix
        self.lambda_ = lambda_
    
    def forward(self, current_loss):
        ewc_loss = 0
        for name, param in self.model.named_parameters():
            if name in self.fisher_matrix:
                ewc_loss += torch.sum(self.fisher_matrix[name] * (param - self.old_params[name]).pow(2))
        total_loss = current_loss + self.lambda_ * ewc_loss
        return total_loss

def compute_fisher_information(model, dataloader, device):
    fisher_matrix = {}
    for name, param in model.named_parameters():
        fisher_matrix[name] = torch.zeros_like(param)

    model.eval()
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        model.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        for name, param in model.named_parameters():
            fisher_matrix[name] += param.grad.pow(2)

    for name in fisher_matrix:
        fisher_matrix[name] = fisher_matrix[name] / len(dataloader)

    return fisher_matrix

def save_old_params(model):
    old_params = {}
    for name, param in model.named_parameters():
        old_params[name] = param.clone().detach()
    return old_params

# Function to fine-tune the model with EWC
def fine_tune_with_ewc(model, train_dataset, val_dataset, output_dir, old_params=None, fisher_matrix=None):
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,  # Reduced batch size to fit in memory
        per_device_eval_batch_size=4,   # Reduced batch size to fit in memory
        evaluation_strategy="epoch",
        save_steps=1000,
        save_total_limit=2,
        logging_dir='./logs',
        gradient_accumulation_steps=2,  # Accumulate gradients to simulate larger batch size
        fp16=True,  # Enable mixed precision training to save memory
    )
    
    optimizer = optim.AdamW(model.parameters(), lr=5e-5)

    if old_params and fisher_matrix:
        ewc_loss = EWCLoss(model, old_params, fisher_matrix, lambda_=0.4)

    # Use the custom collate function
    train_dataloader = DataLoader(train_dataset, batch_size=training_args.per_device_train_batch_size, collate_fn=custom_collate_fn)

    for epoch in range(training_args.num_train_epochs):
        model.train()
        model.to(device)  # Ensure model is on the correct device
        torch.cuda.empty_cache()  # Clear cache to free up memory
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}  # Ensure batch is on the correct device
            outputs = model(**batch)
            loss = outputs.loss

            if old_params and fisher_matrix:
                loss = ewc_loss(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return model

# Function to evaluate the model using BLEU score
def evaluate_model(model, test_dataset, device):
    model.to(device)  # Ensure model is on the correct device
    torch.cuda.empty_cache()  # Clear cache to free up memory
    test_results = []
    
    for example in test_dataset:
        inputs = tokenizer(example['func_code_string'], return_tensors="pt", max_length=512, truncation=True).to(device)
        outputs = model.generate(**inputs)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        reference = example['func_documentation_string']
        test_results.append({"prediction": prediction, "reference": reference})
    
    predictions = [result["prediction"] for result in test_results]
    references = [nltk.word_tokenize(result["reference"]) for result in test_results]
    tokenized_predictions = [nltk.word_tokenize(pred) for pred in predictions]
    
    bleu = nltk.translate.bleu_score.corpus_bleu([[ref] for ref in references], tokenized_predictions)
    
    return {"bleu": bleu}

# Define the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained CodeT5 model
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')

# Step 1: Fine-tune on Java
model = fine_tune_with_ewc(model, train_java, val_java, './results_java')
fisher_matrix_java = compute_fisher_information(model, DataLoader(val_java, batch_size=4, collate_fn=custom_collate_fn), device)
old_params_java = save_old_params(model)

# Evaluate on all languages after training on Java
java_results_after_java = evaluate_model(model, test_java, device)
python_results_after_java = evaluate_model(model, test_python, device)
go_results_after_java = evaluate_model(model, test_go, device)

# Step 2: Fine-tune on Python with EWC
model = fine_tune_with_ewc(model, train_python, val_python, './results_python', old_params_java, fisher_matrix_java)
fisher_matrix_python = compute_fisher_information(model, DataLoader(val_python, batch_size=4, collate_fn=custom_collate_fn), device)
old_params_python = save_old_params(model)

# Evaluate on all languages after training on Python
java_results_after_python = evaluate_model(model, test_java, device)
python_results_after_python = evaluate_model(model, test_python, device)
go_results_after_python = evaluate_model(model, test_go, device)

# Step 3: Fine-tune on Go with EWC
model = fine_tune_with_ewc(model, train_go, val_go, './results_go', old_params_python, fisher_matrix_python)

# Evaluate on all languages after training on Go
java_results_after_go = evaluate_model(model, test_java, device)
python_results_after_go = evaluate_model(model, test_python, device)
go_results_after_go = evaluate_model(model, test_go, device)

# Display results after training on Java
print("Results after training on Java:")
print("Java Test Results:", java_results_after_java)
print("Python Test Results:", python_results_after_java)
print("Go Test Results:", go_results_after_java)

# Display results after training on Python
print("\nResults after training on Python:")
print("Java Test Results:", java_results_after_python)
print("Python Test Results:", python_results_after_python)
print("Go Test Results:", go_results_after_python)

# Display results after training on Go
print("\nResults after training on Go:")
print("Java Test Results:", java_results_after_go)
print("Python Test Results:", python_results_after_go)
print("Go Test Results:", go_results_after_go)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.

Results after training on Java:
Java Test Results: {'bleu': 0.026341522383927164}
Python Test Results: {'bleu': 0.009722728672314655}
Go Test Results: {'bleu': 0.02905000989726947}

Results after training on Python:
Java Test Results: {'bleu': 0.001360040253230163}
Python Test Results: {'bleu': 0.022398872216613276}
Go Test Results: {'bleu': 0.0011622314351092589}

Results after training on Go:
Java Test Results: {'bleu': 0.0023664290539881752}
Python Test Results: {'bleu': 0.00622360050249214}
Go Test Results: {'bleu': 0.10808743104359486}


In [ ]:
#1000 codes, ewc, normal bleu

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, TrainingArguments, default_data_collator
import nltk
from nltk.translate.bleu_score import SmoothingFunction

# Ensure necessary downloads for BLEU metric
nltk.download('punkt')

# Custom collate function to handle different data types
def custom_collate_fn(features):
    batch = {}
    for k in features[0].keys():
        if isinstance(features[0][k], list) and isinstance(features[0][k][0], int):  # Tokenized data (assumed to be lists of integers)
            batch[k] = torch.tensor([f[k] for f in features])
        elif isinstance(features[0][k], torch.Tensor):  # Already tensors
            batch[k] = torch.stack([f[k] for f in features])
        else:
            batch[k] = [f[k] for f in features]  # Keep as a list (likely for non-tensor data like strings)
    
    return batch

# EWC Loss function
class EWCLoss(nn.Module):
    def __init__(self, model, old_params, fisher_matrix, lambda_=0.4):
        super(EWCLoss, self).__init__()
        self.model = model
        self.old_params = old_params
        self.fisher_matrix = fisher_matrix
        self.lambda_ = lambda_
    
    def forward(self, current_loss):
        ewc_loss = 0
        for name, param in self.model.named_parameters():
            if name in self.fisher_matrix:
                ewc_loss += torch.sum(self.fisher_matrix[name] * (param - self.old_params[name]).pow(2))
        total_loss = current_loss + self.lambda_ * ewc_loss
        return total_loss

def compute_fisher_information(model, dataloader, device):
    fisher_matrix = {}
    for name, param in model.named_parameters():
        fisher_matrix[name] = torch.zeros_like(param)

    model.eval()
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        model.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        for name, param in model.named_parameters():
            fisher_matrix[name] += param.grad.pow(2)

    for name in fisher_matrix:
        fisher_matrix[name] = fisher_matrix[name] / len(dataloader)

    return fisher_matrix

def save_old_params(model):
    old_params = {}
    for name, param in model.named_parameters():
        old_params[name] = param.clone().detach()
    return old_params

# Function to fine-tune the model with EWC
def fine_tune_with_ewc(model, train_dataset, val_dataset, output_dir, old_params=None, fisher_matrix=None):
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,  # Reduced batch size to fit in memory
        per_device_eval_batch_size=4,   # Reduced batch size to fit in memory
        evaluation_strategy="epoch",
        save_steps=1000,
        save_total_limit=2,
        logging_dir='./logs',
        gradient_accumulation_steps=2,  # Accumulate gradients to simulate larger batch size
        fp16=True,  # Enable mixed precision training to save memory
    )
    
    optimizer = optim.AdamW(model.parameters(), lr=5e-5)

    if old_params and fisher_matrix:
        ewc_loss = EWCLoss(model, old_params, fisher_matrix, lambda_=0.4)

    # Use the custom collate function
    train_dataloader = DataLoader(train_dataset, batch_size=training_args.per_device_train_batch_size, collate_fn=custom_collate_fn)

    for epoch in range(training_args.num_train_epochs):
        model.train()
        model.to(device)  # Ensure model is on the correct device
        torch.cuda.empty_cache()  # Clear cache to free up memory
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}  # Ensure batch is on the correct device
            outputs = model(**batch)
            loss = outputs.loss

            if old_params and fisher_matrix:
                loss = ewc_loss(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return model

# Function to evaluate the model using Smoothed BLEU score
def evaluate_model(model, test_dataset, device):
    model.to(device)  # Ensure model is on the correct device
    torch.cuda.empty_cache()  # Clear cache to free up memory
    test_results = []
    
    for example in test_dataset:
        inputs = tokenizer(example['func_code_string'], return_tensors="pt", max_length=512, truncation=True).to(device)
        outputs = model.generate(**inputs)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        reference = example['func_documentation_string']
        test_results.append({"prediction": prediction, "reference": reference})
    
    predictions = [result["prediction"] for result in test_results]
    references = [nltk.word_tokenize(result["reference"]) for result in test_results]
    tokenized_predictions = [nltk.word_tokenize(pred) for pred in predictions]
    
    # Compute smoothed BLEU scores
    smoothing_function = SmoothingFunction().method1
    bleu = nltk.translate.bleu_score.corpus_bleu([[ref] for ref in references], tokenized_predictions, smoothing_function=smoothing_function)
    
    return {"bleu": bleu}

# Define the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained CodeT5 model
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')

# Step 1: Fine-tune on Java
model = fine_tune_with_ewc(model, train_java, val_java, './results_java')
fisher_matrix_java = compute_fisher_information(model, DataLoader(val_java, batch_size=4, collate_fn=custom_collate_fn), device)
old_params_java = save_old_params(model)

# Evaluate on all languages after training on Java
java_results_after_java = evaluate_model(model, test_java, device)
python_results_after_java = evaluate_model(model, test_python, device)
go_results_after_java = evaluate_model(model, test_go, device)

# Step 2: Fine-tune on Python with EWC
model = fine_tune_with_ewc(model, train_python, val_python, './results_python', old_params_java, fisher_matrix_java)
fisher_matrix_python = compute_fisher_information(model, DataLoader(val_python, batch_size=4, collate_fn=custom_collate_fn), device)
old_params_python = save_old_params(model)

# Evaluate on all languages after training on Python
java_results_after_python = evaluate_model(model, test_java, device)
python_results_after_python = evaluate_model(model, test_python, device)
go_results_after_python = evaluate_model(model, test_go, device)

# Step 3: Fine-tune on Go with EWC
model = fine_tune_with_ewc(model, train_go, val_go, './results_go', old_params_python, fisher_matrix_python)

# Evaluate on all languages after training on Go
java_results_after_go = evaluate_model(model, test_java, device)
python_results_after_go = evaluate_model(model, test_python, device)
go_results_after_go = evaluate_model(model, test_go, device)

# Display results after training on Java
print("Results after training on Java:")
print("Java Test Results:", java_results_after_java)
print("Python Test Results:", python_results_after_java)
print("Go Test Results:", go_results_after_java)

# Display results after training on Python
print("\nResults after training on Python:")
print("Java Test Results:", java_results_after_python)
print("Python Test Results:", python_results_after_python)
print("Go Test Results:", go_results_after_python)

# Display results after training on Go
print("\nResults after training on Go:")
print("Java Test Results:", java_results_after_go)
print("Python Test Results:", python_results_after_go)
print("Go Test Results:", go_results_after_go)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.

Results after training on Java:
Java Test Results: {'bleu': 0.019047554191201493}
Python Test Results: {'bleu': 0.014187685572252473}
Go Test Results: {'bleu': 0.01276626324217596}

Results after training on Python:
Java Test Results: {'bleu': 0.0003615596519135391}
Python Test Results: {'bleu': 0.03070638629396841}
Go Test Results: {'bleu': 0.0001992415818252553}

Results after training on Go:
Java Test Results: {'bleu': 0.004034870505773016}
Python Test Results: {'bleu': 0.011322548164213471}
Go Test Results: {'bleu': 0.058062633590045765}


In [ ]:
#10000 codes, ewc, smoothed bleu